# Analisis Database Postgresql Menggunakan Python 

Sebelum melakukan analisis database postgresql kita harus menginstall library "pyscopg2" yang digunakan
untuk mengakses database postgresql dengan python

pip install psycopg2

Dalam kasus ini kita menggunakan database dvdrental dari PostgreSQL. 

Setelah library berhasil terinstal maka kita terlebih dahulu membuat kode program untuk 
menghubungkan antara Python dan PostgreSQL. Berikut kode program di bawah: 

In [15]:
import psycopg2
from psycopg2 import Error

try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5433",
                                  database="iykra-sql")

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute("SELECT version();")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'iykra-sql', 'host': '127.0.0.1', 'port': '5433', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  ('PostgreSQL 13.2 on x86_64-apple-darwin, compiled by Apple clang version 11.0.3 (clang-1103.0.32.59), 64-bit',) 

PostgreSQL connection is closed


Untuk menghubungkan antara Python dan PostgreSQL kita harus mendefinisikan database Postgre yang akan kita gunakan. 
Libary "psycopg2" akan melakukan inisialisasi database tersebut ke dalam program python.

Menjalankan query SQL dilakukan menggunakan fungsi try - except 

# Pertanyaan:

Berikut ini adalah pertanyaan untuk analisis database dvdrental menggunakan Python dan PostgreSQL

### 1.  I wonder, how many films have a rating of "R" and a replacement cost between \$5 and \\$10?

In [16]:
import psycopg2
from psycopg2 import Error
import pandas as pd
try:
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5433",
                                  database="iykra-sql")
    cursor = connection.cursor()
    postgreSQL_select_Query = (""" SELECT title,replacement_cost, rating 
    FROM public.film  WHERE rating = 'R'
    AND replacement_cost BETWEEN 5 AND 10 ORDER BY film_id ASC """)
    cursor.execute(postgreSQL_select_Query)
    result_ = pd.read_sql(postgreSQL_select_Query, connection)
    print(result_, "\n")
    
except (Exception, Error) as error:
    print("Error while fetching to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

                    title  replacement_cost rating
0    Anaconda Confessions              9.99      R
1  Deliverance Mulholland              9.99      R
2           Kissing Dolls              9.99      R
3          Pluto Oleander              9.99      R
4            Purple Movie              9.99      R
5           Roxanne Rebel              9.99      R
6         Sun Confessions              9.99      R 

PostgreSQL connection is closed


### 2. We have two staff members with staff IDs 1 and 2. We want to give a bonus to the staff member that handled the most payments. How many payments did each staff member handle? And how much was the total amount processed by each staff member

In [7]:
import psycopg2
from psycopg2 import Error
import pandas as pd
try:
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5433",
                                  database="iykra-sql")
    cursor = connection.cursor()
    postgreSQL_select_Query = ("""SELECT staff_id, SUM(amount) AS total, COUNT (customer_id) 
    AS total_member_handled FROM  public.payment GROUP BY staff_id""")
    cursor.execute(postgreSQL_select_Query)
    result_ = pd.read_sql(postgreSQL_select_Query, connection)
    print(result_, "\n")
except (Exception, Error) as error:
    print("Error while fetching to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

   staff_id     total  total_member_handled
0         1  30252.12                  7292
1         2  31059.92                  7304 

PostgreSQL connection is closed


Jumlah member paling banyak dilayani oleh staff_id 2 dengan total 7304 member 

### 3. Corporate headquarters is auditing the store, they want to know the average replacement cost of movies by rating

In [8]:
import psycopg2
from psycopg2 import Error
import pandas as pd
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5433",
                                  database="iykra-sql")
    cursor = connection.cursor()
    postgreSQL_select_Query = (""" SELECT AVG (replacement_cost)AS avg_replacement_cost, rating FROM film
GROUP BY rating
""")
    cursor.execute(postgreSQL_select_Query)
    result_ = pd.read_sql(postgreSQL_select_Query, connection)
    print(result_, "\n")
except (Exception, Error) as error:
    print("Error while fetching to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

   avg_replacement_cost rating
0             20.231026      R
1             20.137619  NC-17
2             20.124831      G
3             18.959072     PG
4             20.402556  PG-13 

PostgreSQL connection is closed


### 4. We want to send coupons to the 5 customers who have spent the most amount of money. Get the customer name, emal and their spent amount!

In [3]:
import psycopg2
from psycopg2 import Error
import pandas as pd
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5433",
                                  database="iykra-sql")
    cursor = connection.cursor()
    postgreSQL_select_Query = (""" SELECT
	customer_id,
	first_name,
	last_name,
	email,
	SUM(amount)AS total_amount
FROM
	payment
INNER JOIN customer USING(customer_id)
GROUP BY customer_id 
ORDER BY total_amount DESC
LIMIT 5;

""")
    cursor.execute(postgreSQL_select_Query)
    result_ = pd.read_sql(postgreSQL_select_Query, connection)
    print(result_, "\n")
except (Exception, Error) as error:
    print("Error while fetching to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

   customer_id first_name last_name                              email  \
0          148    Eleanor      Hunt    eleanor.hunt@sakilacustomer.org   
1          526       Karl      Seal       karl.seal@sakilacustomer.org   
2          178     Marion    Snyder   marion.snyder@sakilacustomer.org   
3          137     Rhonda   Kennedy  rhonda.kennedy@sakilacustomer.org   
4          144      Clara      Shaw      clara.shaw@sakilacustomer.org   

   total_amount  
0        211.55  
1        208.58  
2        194.61  
3        191.62  
4        189.60   

PostgreSQL connection is closed


### 5. A customer wants to know the films about ‘ancient’. How many recommendations could you give for him?

In [13]:
import psycopg2
from psycopg2 import Error
import pandas as pd
try:
    # Connect to an existing database
    connection = psycopg2.connect(user="postgres",
                                  password="admin",
                                  host="127.0.0.1",
                                  port="5433",
                                  database="iykra-sql")
    cursor = connection.cursor()
    postgreSQL_select_Query = (""" SELECT title, fulltext FROM film
    WHERE fulltext @@ to_tsquery('ancient'); """)
                              
    query_ = ("""SELECT COUNT(title) AS title_count
                              FROM film
                              WHERE fulltext @@ to_tsquery ('ancient');""")
    cursor.execute(postgreSQL_select_Query)
    result_ = pd.read_sql(postgreSQL_select_Query, connection)
    result_2 = pd.read_sql(query_, connection)
    print(result_, "\n")
    print(result_2, "\n")
except (Exception, Error) as error:
    print("Error while fetching to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

                   title                                           fulltext
0        Airport Pollock  'airport':1 'ancient':18 'confront':14 'epic':...
1         Ace Goldfinger  'ace':1 'administr':9 'ancient':19 'astound':4...
2           Agent Truman  'agent':1 'ancient':19 'boy':11 'china':20 'es...
3       Aladdin Calendar  'action':5 'action-pack':4 'aladdin':1 'ancien...
4     Antitrust Tomatoes  'administr':17 'ancient':19 'antitrust':1 'dat...
..                   ...                                                ...
119      Whisperer Giant  'ancient':18 'confront':14 'dentist':8 'giant'...
120            Wife Turn  'ancient':20 'awe':5 'awe-inspir':4 'confront'...
121            Wonka Sea  'ancient':19 'boat':8 'brilliant':4 'india':20...
122  Working Microcosmos  'ancient':18 'china':19 'dentist':8 'dog':11 '...
123    Zoolander Fiction  'ancient':19 'boat':11 'china':20 'discov':14 ...

[124 rows x 2 columns] 

   title_count
0          124 

PostgreSQL connection is close

Terdapat 124 rekomendasi film dengan fulltext 'ancient'